In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 51.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8f454d7f35fecd55e39f2051dd8f1a3cfb28c543ba7212d70de8879986b83251
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,622 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,937 kB]
Hit:13 http://ppa.la

In [5]:
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
##import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import datetime as dt
import haversine as hs
from haversine import Unit
from pyspark.sql.functions import lit
from pyspark.sql.functions import isnan, when, count, col
ID=0


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Team-3-Project").getOrCreate()

In [7]:
from pyspark import SparkFiles
url = "https://team-3-s3-data-bucket.s3.amazonaws.com/202111-citibike-tripdata.csv"
spark.sparkContext.addFile(url)
bikedata_df = spark.read.csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
bikedata_df.head(5)

[Row(ride_id='F8C750DE34200458', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 8, 7, 34, 45), ended_at=datetime.datetime(2021, 11, 8, 7, 52, 14), start_station_name='Franklin Ave & St Marks Ave', start_station_id='4107.05', end_station_name='Carroll St & Smith St', end_station_id='4225.14', start_lat=40.6758324, start_lng=-73.9561677, end_lat=40.680611, end_lng=-73.99475825, member_casual='member'),
 Row(ride_id='150B96183EA723BE', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 24, 10, 27, 6), ended_at=datetime.datetime(2021, 11, 24, 10, 34, 40), start_station_name='Pleasant Ave & E 120 St', start_station_id='7579.01', end_station_name='Willis Ave & E 137 St', end_station_id='7726.02', start_lat=40.7974772, start_lng=-73.9311847, end_lat=40.808384, end_lng=-73.923604, member_casual='member'),
 Row(ride_id='55BA8CE0120B8994', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 27, 18, 11, 36), ended_at=datetime.datetime(20

In [8]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-16 23:22:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.12MB/s    in 0.2s    

2022-08-16 23:22:36 (6.12 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [9]:
bikedata_df.show()


+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------+--------------+------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------+--------------+------------+--------------+-------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|         4107.05|Carroll St & Smit...|       4225.14| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|
|150B96183EA723BE|electric_bike|2021-11-24 10:27:06|2021-11-24 10:34:40|Pleasant Ave & E ...|         7579.01|Willis Ave & E 13...|     

In [10]:

new_bikedata_df = bikedata_df.drop('start_station_id', 'end_station_id')

In [11]:
new_bikedata_df.show()

+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|    end_station_name|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|Carroll St & Smit...| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|
|150B96183EA723BE|electric_bike|2021-11-24 10:27:06|2021-11-24 10:34:40|Pleasant Ave & E ...|Willis Ave & E 13...| 40.7974772|   -73.9311847|   40.808384|    -73.923604|       member|
|55BA8CE0120B8994|electric_bike|2021-11-27 18:11:36|2021-11-27 19:19:16|West St 

In [12]:

#!pip install haversine

new_bikedata_df=new_bikedata_df.withColumn("distance_traveled", lit(0)) 
new_bikedata_df.show()




+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|    end_station_name|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|distance_traveled|
+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|Carroll St & Smit...| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|                0|
|150B96183EA723BE|electric_bike|2021-11-24 10:27:06|2021-11-24 10:34:40|Pleasant Ave & E ...|Willis Ave & E 13...| 40.7974772|   -73.9311847|   40.808384|    -73.923604|       member|         

In [13]:
new_bikedata_df.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'end_station_name',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual',
 'distance_traveled']

In [14]:
##Add new columns for POSIX timestamps
new_bikedata_df = new_bikedata_df.withColumn("started_at_sec", new_bikedata_df["started_at"])
new_bikedata_df = new_bikedata_df.withColumn("ended_at_sec", new_bikedata_df["ended_at"])
new_bikedata_df.show()

+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+-------------------+-------------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|    end_station_name|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|distance_traveled|     started_at_sec|       ended_at_sec|
+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+-------------------+-------------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|Carroll St & Smit...| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|                0|2021-11-08 07:34:45|2021-11-08 07:52:14|
|150B96183EA723BE|electric_bike|

In [15]:
# COLUMNS
new_bikedata_df.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'end_station_name',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual',
 'distance_traveled',
 'started_at_sec',
 'ended_at_sec']

In [16]:
# LIST DATAFRAME DATA TYPES
new_bikedata_df.dtypes

[('ride_id', 'string'),
 ('rideable_type', 'string'),
 ('started_at', 'timestamp'),
 ('ended_at', 'timestamp'),
 ('start_station_name', 'string'),
 ('end_station_name', 'string'),
 ('start_lat', 'double'),
 ('start_lng', 'double'),
 ('end_lat', 'double'),
 ('end_lng', 'double'),
 ('member_casual', 'string'),
 ('distance_traveled', 'int'),
 ('started_at_sec', 'timestamp'),
 ('ended_at_sec', 'timestamp')]

In [17]:
# Find and remove any row with a null value.

new_bikedata_df.filter("ended_at is NULL").show()
new_bikedata_df.filter(new_bikedata_df.ended_at.isNull()).show()
new_bikedata_df.filter(col("ended_at").isNull()).show()
new_bikedata_df.filter("started_at is NULL").show()
new_bikedata_df.filter(new_bikedata_df.started_at.isNull()).show()
new_bikedata_df.filter(col("started_at").isNull()).show()
new_bikedata_df.filter("ride_id is NULL").show()
new_bikedata_df.filter(new_bikedata_df.ride_id.isNull()).show()
new_bikedata_df.filter(col("ride_id").isNull()).show()
new_bikedata_df.filter("rideable_type is NULL").show()
new_bikedata_df.filter(new_bikedata_df.rideable_type.isNull()).show()
new_bikedata_df.filter(col("rideable_type").isNull()).show()
new_bikedata_df.filter("start_station_name is NULL").show()
new_bikedata_df.filter(new_bikedata_df.start_station_name.isNull()).show()
new_bikedata_df.filter(col("start_station_name").isNull()).show()
new_bikedata_df.filter("end_station_name is NULL").show()
new_bikedata_df.filter(new_bikedata_df.end_station_name.isNull()).show()
new_bikedata_df.filter(col("end_station_name").isNull()).show()
new_bikedata_df.filter("start_lat is NULL").show()
new_bikedata_df.filter(new_bikedata_df.start_lat.isNull()).show()
new_bikedata_df.filter(col("start_lat").isNull()).show()
new_bikedata_df.filter("start_lng is NULL").show()
new_bikedata_df.filter(new_bikedata_df.start_lng.isNull()).show()
new_bikedata_df.filter(col("start_lng").isNull()).show()
new_bikedata_df.filter("end_lng is NULL").show()
new_bikedata_df.filter(new_bikedata_df.end_lng.isNull()).show()
new_bikedata_df.filter(col("end_lng").isNull()).show()
new_bikedata_df.filter("end_lat is NULL").show()
new_bikedata_df.filter(new_bikedata_df.end_lat.isNull()).show()
new_bikedata_df.filter(col("end_lat").isNull()).show()
new_bikedata_df.filter("member_casual is NULL").show()
new_bikedata_df.filter(new_bikedata_df.member_casual.isNull()).show()
new_bikedata_df.filter(col("member_casual").isNull()).show()
new_bikedata_df.filter("distance_traveled is NULL").show()
new_bikedata_df.filter(new_bikedata_df.distance_traveled.isNull()).show()
new_bikedata_df.filter(col("distance_traveled").isNull()).show()
new_bikedata_df.filter("ended_at_sec is NULL").show()
new_bikedata_df.filter(new_bikedata_df.ended_at_sec.isNull()).show()
new_bikedata_df.filter(col("ended_at_sec").isNull()).show()
new_bikedata_df.filter("started_at_sec is NULL").show()
new_bikedata_df.filter(new_bikedata_df.started_at_sec.isNull()).show()
new_bikedata_df.filter(col("started_at_sec").isNull()).show()

+-------+-------------+----------+--------+------------------+----------------+---------+---------+-------+-------+-------------+-----------------+--------------+------------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|end_station_name|start_lat|start_lng|end_lat|end_lng|member_casual|distance_traveled|started_at_sec|ended_at_sec|
+-------+-------------+----------+--------+------------------+----------------+---------+---------+-------+-------+-------------+-----------------+--------------+------------+
+-------+-------------+----------+--------+------------------+----------------+---------+---------+-------+-------+-------------+-----------------+--------------+------------+

+-------+-------------+----------+--------+------------------+----------------+---------+---------+-------+-------+-------------+-----------------+--------------+------------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|end_station_name|start_lat|start_lng|end_lat|end_lng|memb

In [18]:
new_bikedata_df.show()

+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+-------------------+-------------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|    end_station_name|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|distance_traveled|     started_at_sec|       ended_at_sec|
+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+-----------------+-------------------+-------------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|Carroll St & Smit...| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|                0|2021-11-08 07:34:45|2021-11-08 07:52:14|
|150B96183EA723BE|electric_bike|

In [19]:
new3_bikedata_df=new_bikedata_df.toPandas()
new3_bikedata_df.head(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,distance_traveled,started_at_sec,ended_at_sec
0,F8C750DE34200458,electric_bike,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,0,2021-11-08 07:34:45,2021-11-08 07:52:14
1,150B96183EA723BE,electric_bike,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,0,2021-11-24 10:27:06,2021-11-24 10:34:40
2,55BA8CE0120B8994,electric_bike,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,0,2021-11-27 18:11:36,2021-11-27 19:19:16
3,BEEAF39A29375A4F,electric_bike,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,0,2021-11-22 14:52:20,2021-11-22 15:30:40
4,A322C249E2152BE8,electric_bike,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,0,2021-11-10 14:32:56,2021-11-10 14:36:19
5,47838257604530BE,electric_bike,2021-11-09 07:20:53,2021-11-09 07:26:13,Franklin Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.675832,-73.956168,40.673134,-73.969106,casual,0,2021-11-09 07:20:53,2021-11-09 07:26:13
6,362E604AA6EECA77,classic_bike,2021-11-03 16:23:08,2021-11-03 16:30:39,Classon Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.676520,-73.959608,40.673134,-73.969106,casual,0,2021-11-03 16:23:08,2021-11-03 16:30:39
7,68CF5DD571217350,classic_bike,2021-11-03 14:00:14,2021-11-03 14:07:12,Columbia St & Degraw St,Bergen St & Smith St,40.685930,-74.002424,40.686744,-73.990632,casual,0,2021-11-03 14:00:14,2021-11-03 14:07:12
8,096AC000442AA859,classic_bike,2021-11-11 16:37:28,2021-11-11 16:50:51,E 32 St & Park Ave,W 45 St & 8 Ave,40.745712,-73.981948,40.759291,-73.988597,member,0,2021-11-11 16:37:28,2021-11-11 16:50:51
9,7ABE54EAAB9C501E,classic_bike,2021-11-17 12:39:03,2021-11-17 12:42:08,President St & Henry St,Carroll St & Smith St,40.682800,-73.999904,40.680611,-73.994758,member,0,2021-11-17 12:39:03,2021-11-17 12:42:08


In [20]:
# FIND DUPLICATE ENTRIES
print(f"Duplicate entries: {new3_bikedata_df.duplicated().sum()}")

Duplicate entries: 0


In [21]:
# Simplify bicycle drive type
def change_string(rideable_type):
    if rideable_type == "electric_bike":
        return 1
    else:
        return 0
    
new3_bikedata_df["rideable_type"] = new3_bikedata_df["rideable_type"].apply(change_string)
new3_bikedata_df.head(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,distance_traveled,started_at_sec,ended_at_sec
0,F8C750DE34200458,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,0,2021-11-08 07:34:45,2021-11-08 07:52:14
1,150B96183EA723BE,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,0,2021-11-24 10:27:06,2021-11-24 10:34:40
2,55BA8CE0120B8994,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,0,2021-11-27 18:11:36,2021-11-27 19:19:16
3,BEEAF39A29375A4F,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,0,2021-11-22 14:52:20,2021-11-22 15:30:40
4,A322C249E2152BE8,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,0,2021-11-10 14:32:56,2021-11-10 14:36:19
5,47838257604530BE,1,2021-11-09 07:20:53,2021-11-09 07:26:13,Franklin Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.675832,-73.956168,40.673134,-73.969106,casual,0,2021-11-09 07:20:53,2021-11-09 07:26:13
6,362E604AA6EECA77,0,2021-11-03 16:23:08,2021-11-03 16:30:39,Classon Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.676520,-73.959608,40.673134,-73.969106,casual,0,2021-11-03 16:23:08,2021-11-03 16:30:39
7,68CF5DD571217350,0,2021-11-03 14:00:14,2021-11-03 14:07:12,Columbia St & Degraw St,Bergen St & Smith St,40.685930,-74.002424,40.686744,-73.990632,casual,0,2021-11-03 14:00:14,2021-11-03 14:07:12
8,096AC000442AA859,0,2021-11-11 16:37:28,2021-11-11 16:50:51,E 32 St & Park Ave,W 45 St & 8 Ave,40.745712,-73.981948,40.759291,-73.988597,member,0,2021-11-11 16:37:28,2021-11-11 16:50:51
9,7ABE54EAAB9C501E,0,2021-11-17 12:39:03,2021-11-17 12:42:08,President St & Henry St,Carroll St & Smith St,40.682800,-73.999904,40.680611,-73.994758,member,0,2021-11-17 12:39:03,2021-11-17 12:42:08


In [22]:
#Simplify Ride IDs
def change_string(ride_id):
    if ride_id != "0":
        global ID
        ID = ID +1
        return ID
    else:
        
        return ID
    
new3_bikedata_df["ride_id"] = new3_bikedata_df["ride_id"].apply(change_string)
new3_bikedata_df.head(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,distance_traveled,started_at_sec,ended_at_sec
0,1,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,0,2021-11-08 07:34:45,2021-11-08 07:52:14
1,2,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,0,2021-11-24 10:27:06,2021-11-24 10:34:40
2,3,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,0,2021-11-27 18:11:36,2021-11-27 19:19:16
3,4,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,0,2021-11-22 14:52:20,2021-11-22 15:30:40
4,5,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,0,2021-11-10 14:32:56,2021-11-10 14:36:19
5,6,1,2021-11-09 07:20:53,2021-11-09 07:26:13,Franklin Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.675832,-73.956168,40.673134,-73.969106,casual,0,2021-11-09 07:20:53,2021-11-09 07:26:13
6,7,0,2021-11-03 16:23:08,2021-11-03 16:30:39,Classon Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.676520,-73.959608,40.673134,-73.969106,casual,0,2021-11-03 16:23:08,2021-11-03 16:30:39
7,8,0,2021-11-03 14:00:14,2021-11-03 14:07:12,Columbia St & Degraw St,Bergen St & Smith St,40.685930,-74.002424,40.686744,-73.990632,casual,0,2021-11-03 14:00:14,2021-11-03 14:07:12
8,9,0,2021-11-11 16:37:28,2021-11-11 16:50:51,E 32 St & Park Ave,W 45 St & 8 Ave,40.745712,-73.981948,40.759291,-73.988597,member,0,2021-11-11 16:37:28,2021-11-11 16:50:51
9,10,0,2021-11-17 12:39:03,2021-11-17 12:42:08,President St & Henry St,Carroll St & Smith St,40.682800,-73.999904,40.680611,-73.994758,member,0,2021-11-17 12:39:03,2021-11-17 12:42:08


In [23]:
#Convert values to appropriate types
new3_bikedata_df['distance_traveled']=new3_bikedata_df.rideable_type.astype('float64')
new3_bikedata_df['rideable_type']=new3_bikedata_df.rideable_type.astype('int16')
new3_bikedata_df['start_station_name']=new3_bikedata_df.start_station_name.astype('string')
new3_bikedata_df['end_station_name']=new3_bikedata_df.end_station_name.astype('string')
new3_bikedata_df['member_casual']=new3_bikedata_df.member_casual.astype('string')
new3_bikedata_df['ride_id']=new3_bikedata_df.ride_id.astype('int16')
new3_bikedata_df['started_at']= pd.to_datetime(new3_bikedata_df['started_at'])
new3_bikedata_df['ended_at']= pd.to_datetime(new3_bikedata_df['ended_at'])
## LIST DATAFRAME DATA TYPES
new3_bikedata_df.dtypes


ride_id                        int16
rideable_type                  int16
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            string
end_station_name              string
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 string
distance_traveled            float64
started_at_sec        datetime64[ns]
ended_at_sec          datetime64[ns]
dtype: object

In [24]:
#Calculate the distance traveled
for x in range (len('ride_id')):
               
    loc1 = (new3_bikedata_df['start_lat'].values[x], new3_bikedata_df['start_lng'].values[x])
    loc2 = (new3_bikedata_df['end_lat'].values[x], new3_bikedata_df['end_lng'].values[x])
    new3_bikedata_df['distance_traveled'].values[x] =hs.haversine(loc1,loc2,unit=Unit.MILES)
    
new3_bikedata_df.head(20)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,distance_traveled,started_at_sec,ended_at_sec
0,1,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,2.048892,2021-11-08 07:34:45,2021-11-08 07:52:14
1,2,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,0.851522,2021-11-24 10:27:06,2021-11-24 10:34:40
2,3,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,3.580215,2021-11-27 18:11:36,2021-11-27 19:19:16
3,4,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,1.674019,2021-11-22 14:52:20,2021-11-22 15:30:40
4,5,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,0.522483,2021-11-10 14:32:56,2021-11-10 14:36:19
5,6,1,2021-11-09 07:20:53,2021-11-09 07:26:13,Franklin Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.675832,-73.956168,40.673134,-73.969106,casual,0.703162,2021-11-09 07:20:53,2021-11-09 07:26:13
6,7,0,2021-11-03 16:23:08,2021-11-03 16:30:39,Classon Ave & St Marks Ave,Plaza St East & Flatbush Ave,40.676520,-73.959608,40.673134,-73.969106,casual,0.549940,2021-11-03 16:23:08,2021-11-03 16:30:39
7,8,0,2021-11-03 14:00:14,2021-11-03 14:07:12,Columbia St & Degraw St,Bergen St & Smith St,40.685930,-74.002424,40.686744,-73.990632,casual,0.000000,2021-11-03 14:00:14,2021-11-03 14:07:12
8,9,0,2021-11-11 16:37:28,2021-11-11 16:50:51,E 32 St & Park Ave,W 45 St & 8 Ave,40.745712,-73.981948,40.759291,-73.988597,member,0.000000,2021-11-11 16:37:28,2021-11-11 16:50:51
9,10,0,2021-11-17 12:39:03,2021-11-17 12:42:08,President St & Henry St,Carroll St & Smith St,40.682800,-73.999904,40.680611,-73.994758,member,0.000000,2021-11-17 12:39:03,2021-11-17 12:42:08


In [26]:
#Save the cleaned data
new3_bikedata_df.to_csv('202111-citibike-tripdata-cleaned.csv', index=False)

In [27]:
new3_bikedata_df.head(100)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,distance_traveled,started_at_sec,ended_at_sec
0,1,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,2.048892,2021-11-08 07:34:45,2021-11-08 07:52:14
1,2,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,0.851522,2021-11-24 10:27:06,2021-11-24 10:34:40
2,3,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,3.580215,2021-11-27 18:11:36,2021-11-27 19:19:16
3,4,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,1.674019,2021-11-22 14:52:20,2021-11-22 15:30:40
4,5,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,0.522483,2021-11-10 14:32:56,2021-11-10 14:36:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,2021-11-18 17:50:19,2021-11-18 18:09:17,Franklin Ave & St Marks Ave,Wythe Ave & Metropolitan Ave,40.675832,-73.956168,40.716887,-73.963198,member,1.000000,2021-11-18 17:50:19,2021-11-18 18:09:17
96,97,1,2021-11-11 16:37:28,2021-11-11 16:55:21,Franklin Ave & St Marks Ave,Wythe Ave & Metropolitan Ave,40.675832,-73.956168,40.716887,-73.963198,member,1.000000,2021-11-11 16:37:28,2021-11-11 16:55:21
97,98,0,2021-11-07 19:54:09,2021-11-07 20:17:35,West St & Liberty St,10 Ave & W 28 St,40.711444,-74.014847,40.750664,-74.001768,casual,0.000000,2021-11-07 19:54:09,2021-11-07 20:17:35
98,99,0,2021-11-09 19:05:48,2021-11-09 19:38:03,W 53 St & 10 Ave,W Broadway & W Houston St,40.766697,-73.990617,40.726378,-74.000472,casual,0.000000,2021-11-09 19:05:48,2021-11-09 19:38:03
